In [1]:
import os
from env_config import set_env_vars

# --- Load environment variables and secrets ---
set_env_vars()

2025-06-07 11:38:19,966 - INFO - Loaded local .env file from /Users/drew_wilkins/Drews_Files/Drew/Python/Localcode/.env
2025-06-07 11:38:19,967 - INFO - Running in context: CLI
2025-06-07 11:38:19,967 - INFO - Force authentication: False


In [2]:
from gcp_utils import get_gcp_credentials, init_sheets_client, init_drive_client
from qdrant_utils import init_qdrant_client

creds = get_gcp_credentials()
sheets_client = init_sheets_client(creds)
drive_client = init_drive_client(creds)
qdrant_client = init_qdrant_client("cloud")

2025-06-07 11:38:21,299 - INFO - file_cache is only supported with oauth2client<4.0.0
2025-06-07 11:38:21,300 - INFO - Initializing Qdrant client in 'cloud' mode...
2025-06-07 11:38:21,666 - INFO - HTTP Request: GET https://0c82e035-1105-40f2-a0bd-ecc44a016f15.us-east4-0.gcp.cloud.qdrant.io:6333 "HTTP/1.1 200 OK"
2025-06-07 11:38:21,738 - INFO - HTTP Request: GET https://0c82e035-1105-40f2-a0bd-ecc44a016f15.us-east4-0.gcp.cloud.qdrant.io:6333/collections "HTTP/1.1 200 OK"
2025-06-07 11:38:21,739 - INFO - Available Qdrant collections: ['ASK_vectorstore', 'ask_pdf_docs', 'ASK_vectorstore-backup21APR2025', 'ask_pdf_pages']
2025-06-07 11:38:21,739 - INFO - ✅ Confirmed Qdrant collection 'ASK_vectorstore' exists


### Run your functions to test in the cells below

In [ ]:
from delete_tagged import delete_tagged


deleted_rows_df = delete_tagged(drive_client, sheets_client, qdrant_client)

In [ ]:
from library_utils import validate_rows_format


valid_df, invalid_df, log_df = validate_rows_format(
    sheets_client, os.environ["LIBRARY_UNIFIED"])
print(f"\nInvalid rows:\n{invalid_df}")

In [3]:
from cleanup_orphans import find_orphans


orphan_rows, orphan_files, orphan_qdrant_records, log_df = find_orphans(
    drive_client, sheets_client, qdrant_client)

print(f"\nOrphan rows:\n{orphan_rows}")
print(f"\nOrphan files:\n{orphan_files}")
print(f"\nOrphan qdrant records:\n{orphan_qdrant_records}")

2025-06-07 11:38:27,545 - INFO - Cataloging PDF_TAGGING for orphan review
2025-06-07 11:38:27,952 - INFO - Cataloging PDF_LIVE for orphan review
2025-06-07 11:38:29,062 - INFO - ⚠️ Found 1 orphans (rows in LIBRARY_UNIFIED but not in Google Drive folders.
2025-06-07 11:38:29,741 - INFO - ⚠️ Found 18 orphans (files in Google Drive folders but not in LIBRARY_UNIFIED.
2025-06-07 11:38:30,510 - INFO - HTTP Request: POST https://0c82e035-1105-40f2-a0bd-ecc44a016f15.us-east4-0.gcp.cloud.qdrant.io:6333/collections/ASK_vectorstore/points/scroll "HTTP/1.1 200 OK"
2025-06-07 11:38:32,911 - INFO - Retrieving all pdf_ids from Qdrant collection. Found 211 unique pdf_ids.
2025-06-07 11:38:32,917 - WARNING - Found 1 orphans (records in Qdrant but not in LIBRARY_UNIFIED):
 ['d354afd1-95ea-5489-9df7-3abdd322d73c']
2025-06-07 11:38:32,918 - INFO - Returning 20 total log entries.



Orphan rows:
                                                 title  \
166  COAST GUARD EXTERNAL AFFAIRS MANUAL, COMDTINST...   

                                   pdf_id   publication_number organization  \
166  d354afd1-95ea-5489-9df7-3abdd322d73c  COMDTINST M5700.13A       CG-092   

        scope unit            issue_date upsert_date       expiration_date  \
166  National       2021-05-12T00:00:00Z              2031-05-13T00:00:00Z   

    aux_specific public_release  \
166        FALSE           TRUE   

                                     pdf_file_name               embedding  \
166  CG Public Affairs CIM_5700_13A_2021-05-01.pdf  text-embedding-ada-002   

    google_id link      status status_timestamp  
166                 orphan_row                   

Orphan files:
                                    ID  \
16   1R6DtX_1jTKf3WEDARSmW2Ad7WbbxuClG   
40   1C_VU-mFfnEovVaSFeqz0EhDvcPMvqjwa   
41   1xO12dBYJBxVP7XBqBIRpYYN4dHynmRhh   
43   19Ql6ALvx6LEGlYTdOgWTEzosc5LnWCXq   
